In [ ]:
from src.graph_utils import *
from src.ace import *
import os
import time

In [ ]:
base_path = 'results/'

n_shots = 65536

optimization_iteration_count = 0

start_layers = 1
max_layers = 1
initial_params_seed = 123
scipy_optimizer_methods = ["COBYLA", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP"]

heights = [2,4,8]
seeds = [111,222,333,444,555]

In [ ]:
for seed in seeds:
  report_filename = base_path + 'AncillaBasisStateEncoding_newCost_' +  str(seed) + '_' + str(n_shots) + '.txt'
  for height in heights:
    width = height
    print(f'height: {height}, width: {width}, n: {height*width}')
    np.random.seed(seed=seed)
    G, image = generate_problem_instance(height, width)
    print("Image Generated: ",image)
    for scipy_optimizer_method in scipy_optimizer_methods:
      print("Maximum number of layers :", max_layers)
      for n_layers in range(start_layers, max_layers+1,1):
        best_cost = 1000000
        best_cost_binary_solution = [0] * (height*width)
        nc = len(G.nodes())
        nr = ceil(log2(nc))
        nq = nr + 1
        initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
        print(f"Executing QC with {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
        # try:
        start_time = time.time()
        success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value = adaptive_cost_encoding_de(G,
                                                                                                                         initial_params,
                                                                                                                         n_layers = n_layers,
                                                                                                                         optimizer_method = scipy_optimizer_method)
        minimal_encoding_tte = (time.time() - start_time)
        print("New NISQ done for",scipy_optimizer_method,"optimizer with a success status :", success_flag)
        print(f"Appending the results of {height}*{height} image using QC with {n_layers} layers and {scipy_optimizer_method} optimizer.")
        row = []
        row.append(int(G.number_of_nodes()))
        row.append(int(height))
        row.append(int(width))
        row.append(success_flag)
        row.append(''.join(map(str, map(int, (minimal_encoding_solution)))))
        row.append(np.round(minimal_encoding_tte,6))
        row.append(n_layers)
        row.append(np.round(minimal_encoding_cut_value,4))
        row.append(np.round(minimal_encoding_value,4))
        row.append(optimization_iteration_count)
        row.append(scipy_optimizer_method)
        report_file_obj = open(os.path.join(report_filename),'a+')
        report_file_obj.write('__'.join(map(str,row))+'\n')
        report_file_obj.close()
    print('\n')